In [1]:
import yt
from yt import YTArray
from yt import YTQuantity
from yt.data_objects.level_sets.api import *

from astropy import constants as const

import numpy as np

import matplotlib
%matplotlib inline
import matplotlib.pylab as plt
from matplotlib.colors import SymLogNorm, LogNorm

import palettable
import plotting_tools as pt
import yt_functions as ytf





yt : [INFO     ] 2020-05-19 16:15:20,241 Parameters: current_time              = 0.39999990918174
yt : [INFO     ] 2020-05-19 16:15:20,242 Parameters: domain_dimensions         = [ 64  64 256]
yt : [INFO     ] 2020-05-19 16:15:20,242 Parameters: domain_left_edge          = [-0.5 -0.5 -2. ]
yt : [INFO     ] 2020-05-19 16:15:20,243 Parameters: domain_right_edge         = [0.5 0.5 2. ]
yt : [INFO     ] 2020-05-19 16:15:20,244 Parameters: cosmological_simulation   = 0.0


In [ ]:
def find_cold_clumps(ds):
    ad = ds.all_data()
    ad_slice = ds.r[:, :, 0.8:1.2]
    cold_mask = ad_slice[('gas', 'temperature')] < 3.33e5
    rho_cold = ad_slice[('gas', 'density')][cold_mask]
    rho_cold_ave = np.mean(rho_cold)
    rho_cold_std = np.std(rho_cold)

    field = 'density'
    master_clump = Clump(ad, ('gas', field))

    master_clump.add_validator("min_cells", 20)
    # notes for tomorrow:                                                                                                                                   
    # try finding clumps using inverse temperature or negative!                                                                                             
    # maybe try adding validator for temperature or density cutoffs                                                                                         
    # idea: contour around 1 sigma of average cold temperature                                                                                              
    # idea: filter out leafs (especially larger ones... what's going on there...                                                                            
    #        maybe i should change step back down to something  small t oallow smaller clumps                                                               
    # quantify  shape of clump                                                                                                                              
    #master_clump.add_validator("max_temperature")                                                                                                          
    #idea                                                                                                                                                   
    master_clump.add_info_item("total_cells")


    #c_min = YTQuantity(1e-27, 'g/cm**3')#                                                                                                                  
    #c_max = ad["gas", field].max()                                                                                                                         
    #c_max = YTQuantity(-4e4, 'K')                                                                                                                          
    #c_min = YTQuantity(-3e5, 'K')#ad["gas", field].max()                                                                                                   
    c_min = rho_cold_ave - 0.5*rho_cold_std
    c_max = ad["gas", field].max()
    print(c_min, c_max)

    step = 1000.0
    find_clumps(master_clump, c_min, c_max, step);

    leaf_clumps = master_clump.leaves
    return master_clump


In [155]:
def get_masked_data_source(ds, z_min = 0.8, z_max = 1.2, T_cold = 3.33333e5):
    ad = ds.all_data()
    z_abs_code = np.abs(ad[('gas', 'z')] / ds.length_unit.in_units('kpc'))
    zmask= (z_abs_code >= z_min) & (z_abs_code <= z_max)
    return ad[zmask]

def _maximum_gas_temperature(clump, max_temp):
    return (clump["gas", "temperature"].sum() <= max_temp)
add_validator("maximum_gas_temperature", _maximum_gas_temperature)

def _maximum_ave_temperature(clump, max_temp):
    return (np.mean(clump["gas", "temperature"]) <= max_temp)
add_validator("maximum_ave_temperature", _maximum_ave_temperature)
def _log_rho(field, data):
    log_rho = np.log10(data[('gas', 'density')].in_units('g/cm**3'))
    return log_rho
def _log_T(field, data):
    log_rho = -np.log10(data[('gas', 'temperature')])
    return log_rho


In [193]:
def find_cold_clumps(ds):
    ds.add_field(('gas', 'log_rho'), function = _log_rho, units = '')#,overwrite = True)
    ds.add_field(('gas', 'log_T'), function = _log_T, units = '')#,overwrite = True)

    ad = ds.all_data()
    ad_slice = ds.r[:, :, 0.8:1.2]

    cold_mask = ad_slice[('gas', 'temperature')] <= 3.33e5
    hot_mask = ad_slice[('gas', 'temperature')] > 3.33e5

    T_cold = -1*ad_slice[('gas', 'log_T')][cold_mask]
    T_cold_ave = np.mean(T_cold)
    T_cold_std = np.std(T_cold)

    field = 'log_T'
    master_clump = Clump(ad, ('gas', field))

    master_clump.add_validator("min_cells", 3)                                                                                                                                                
    master_clump.add_info_item("total_cells")

    c_min = -5
    #c_min = -(T_cold_ave + T_cold_std)
    #print(c_min)
    c_max = ad["gas", field].max()
    print(c_min, c_max)

#    step = -.001 #1000
    step = -0.1
    find_clumps(master_clump, c_min, c_max, step);

    leaf_clumps = master_clump.leaves
    return master_clump


def find_radius(leaf_clumps):
    radius_list = []
    n_clumps = 0
    H = 43.8535772
    zmin = .8 * H
    zmax = 1.2 * H
    for i, clump in enumerate(leaf_clumps):
        z = np.abs(clump[('gas', 'z')].in_units('kpc'))
        z_ave = np.mean(z)
        danger_zone = (z < 5) | (z > 83)
        #if len(z[danger_zone]) > 0:
        #    print(i, "danger zone", len(z[danger_zone]) / len(z))
        #    continue
        if (z_ave > zmin) and (z_ave < zmax):
            n_clumps += 1
            ncells = len(z)
            radius = np.power(ncells, 1./3.)
            print(i, ncells, radius)
            radius_list.append(radius)

    return n_clumps, np.mean(radius_list), np.median(radius_list), np.std(radius_list)

def plot_clumps(ds, leaf_clumps):
    field = 'temperature'
    p = yt.ProjectionPlot(ds, 'x', ('gas', field), weight_field = 'density')#, 
                         # center = (0, 0, 1), width = (1, 2), window_size = 6)
    p.set_zlim(('gas', field), 1e5, 3e6)
    p.set_cmap(('gas', field), palettable.scientific.sequential.LaJolla_20_r.mpl_colormap)
    p.annotate_clumps(leaf_clumps)
    p.save('plots/%s_%s.png'%(ds.fullpath[31:-7], field))
    
    field = 'density'
    p = yt.ProjectionPlot(ds, 'x', ('gas', field), weight_field = 'density')#, 
#                          center = (0, 0, 1), width = (1, 2), window_size = 6)
    p.set_zlim(('gas', field), 1e-28, 5e-27)
    p.set_cmap(('gas', field), palettable.cmocean.sequential.Tempo_20.mpl_colormap)
    p.annotate_clumps(leaf_clumps)
    p.save('plots/%s_%s.png'%(ds.fullpath[31:-7], field))
        
def clump_finder(sim_name):
    ds = yt.load('../../../simulations/%s/DD0040/DD0040'%sim_name)
    master_clump = find_cold_clumps(ds)
    plot_clumps(ds, master_clump.leaves)
    n_clumps, mean_r, median_r, std_r = find_radius(master_clump.leaves)
    return n_clumps, mean_r, median_r, std_r


In [194]:
res = clump_finder('isocool_tctf_0.1_beta_100.0')
res2 = clump_finder('isocool_tctf_0.3_beta_100.0')
res3 = clump_finder('isocool_tctf_0.3_beta_100.0_cr_1.0')

yt : [INFO     ] 2020-05-20 17:17:25,989 Parameters: current_time              = 0.39999990918174
yt : [INFO     ] 2020-05-20 17:17:25,990 Parameters: domain_dimensions         = [ 64  64 256]
yt : [INFO     ] 2020-05-20 17:17:25,990 Parameters: domain_left_edge          = [-0.5 -0.5 -2. ]
yt : [INFO     ] 2020-05-20 17:17:25,991 Parameters: domain_right_edge         = [0.5 0.5 2. ]
yt : [INFO     ] 2020-05-20 17:17:25,992 Parameters: cosmological_simulation   = 0.0
yt : [INFO     ] 2020-05-20 17:17:25,995 Gathering a field list (this may take a moment.)
/Users/irynabutsky/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Because 'sampling_type' not specified, yt will assume a cell 'sampling_type'
  
/Users/irynabutsky/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Because 'sampling_type' not specified, yt will assume a cell 'sampling_type'
  This is separate from the ipykernel package so we can avoid doing imports until
yt : [IN

-5 -4.600962095385032 dimensionless


yt : [INFO     ] 2020-05-20 17:17:27,123 Linked.
yt : [INFO     ] 2020-05-20 17:17:27,925 Investigating 198 children.
yt : [INFO     ] 2020-05-20 17:17:27,926 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,927 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,928 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,928 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,929 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,929 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,930 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,930 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, st

yt : [INFO     ] 2020-05-20 17:17:27,970 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,970 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,971 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,971 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,972 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,972 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,973 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,973 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:27,973 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0

yt : [INFO     ] 2020-05-20 17:17:28,006 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:28,007 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:28,007 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:28,008 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:28,008 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:28,009 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:28,009 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:28,009 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:28,010 Finding clumps: min: 5.000000e-01, max: -4.600962e+00, step: -0

...

/Users/irynabutsky/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Because 'sampling_type' not specified, yt will assume a cell 'sampling_type'
  
/Users/irynabutsky/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Because 'sampling_type' not specified, yt will assume a cell 'sampling_type'
  This is separate from the ipykernel package so we can avoid doing imports until
yt : [INFO     ] 2020-05-20 17:17:42,864 Finding clumps: min: -5.000000e+00, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:42,927 Linking node (1) contours.
yt : [INFO     ] 2020-05-20 17:17:43,206 Linked.


-5 -4.782803825253598 dimensionless


yt : [INFO     ] 2020-05-20 17:17:43,387 Investigating 35 children.
yt : [INFO     ] 2020-05-20 17:17:43,389 Finding clumps: min: 5.000000e-01, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:43,389 Finding clumps: min: 5.000000e-01, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:43,390 Finding clumps: min: 5.000000e-01, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:43,391 Finding clumps: min: 5.000000e-01, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:43,391 Finding clumps: min: 5.000000e-01, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:43,392 Finding clumps: min: 5.000000e-01, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:43,392 Finding clumps: min: 5.000000e-01, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:43,392 Finding clumps: min: 5.000000e-01, max: -4.782804e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 17:17:43

yt : [INFO     ] 2020-05-20 17:17:45,309 Pixelizing contour 16
yt : [INFO     ] 2020-05-20 17:17:45,311 Pixelizing contour 17
yt : [INFO     ] 2020-05-20 17:17:45,313 Pixelizing contour 18
yt : [INFO     ] 2020-05-20 17:17:45,316 Pixelizing contour 19
yt : [INFO     ] 2020-05-20 17:17:45,319 Pixelizing contour 20
yt : [INFO     ] 2020-05-20 17:17:45,323 Pixelizing contour 21
yt : [INFO     ] 2020-05-20 17:17:45,325 Pixelizing contour 22
yt : [INFO     ] 2020-05-20 17:17:45,328 Pixelizing contour 23
yt : [INFO     ] 2020-05-20 17:17:45,330 Pixelizing contour 24
yt : [INFO     ] 2020-05-20 17:17:45,332 Pixelizing contour 25
yt : [INFO     ] 2020-05-20 17:17:45,334 Pixelizing contour 26
yt : [INFO     ] 2020-05-20 17:17:45,343 Pixelizing contour 27
yt : [INFO     ] 2020-05-20 17:17:45,348 Pixelizing contour 28
yt : [INFO     ] 2020-05-20 17:17:45,352 Pixelizing contour 29
yt : [INFO     ] 2020-05-20 17:17:45,354 Pixelizing contour 30
yt : [INFO     ] 2020-05-20 17:17:45,356 Pixelizing con

4 787 9.23261893131873
9 367 7.159598824832771
10 759 9.121800967815444
27 677 8.78070842816116
29 651 8.66683102909851
31 237 6.188462762136203


In [195]:
# NOTE: want to calculate r ~ c_s*t_cool at a fixed T. --> t_cool ~ T (but density isn't necessarily the same for all the clumps)
print(res)
print(res2)
print(res3)

(40, 4.884177736238145, 4.160167646103808, 2.6758010333907842)
(16, 7.438094508008471, 7.943852176301574, 2.3548995967312276)
(6, 8.191670157227136, 8.723769728629836, 1.1254238251112514)


In [142]:
c = find_cold_clumps(ds)

/Users/irynabutsky/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Because 'sampling_type' not specified, yt will assume a cell 'sampling_type'
  
/Users/irynabutsky/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Because 'sampling_type' not specified, yt will assume a cell 'sampling_type'
  This is separate from the ipykernel package so we can avoid doing imports until
yt : [INFO     ] 2020-05-20 15:19:59,907 Finding clumps: min: -5.000000e+00, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:19:59,952 Linking node (1) contours.


-5 -4.654870961011894 dimensionless


yt : [INFO     ] 2020-05-20 15:20:00,231 Linked.
yt : [INFO     ] 2020-05-20 15:20:00,644 Investigating 93 children.
yt : [INFO     ] 2020-05-20 15:20:00,645 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,645 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,646 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,646 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,647 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,648 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,648 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,649 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, ste

yt : [INFO     ] 2020-05-20 15:20:00,687 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,688 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,688 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,689 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,689 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,690 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,690 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,691 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0.100000
yt : [INFO     ] 2020-05-20 15:20:00,692 Finding clumps: min: 5.000000e-01, max: -4.654871e+00, step: -0

In [147]:
ad = ds.all_data()
ad[('gas', 'temperature')] *= -1

In [148]:
ad[('gas', 'temperature')]

unyt_array([-336059.83459765, -348579.4299501 , -361963.75314972, ...,
            -362615.87765114, -350540.36852038, -338102.85872902], 'K')

In [183]:
(ds.domain_right_edge- ds.domain_left_edge).in_units('kpc')

unyt_array([ 43.8535772 ,  43.8535772 , 175.41430879], 'kpc')